In [22]:
import numpy as np
import pandas as pd
from numba import jit, cuda
from KernelPerceptron import calculate_accuracy, kernel_perceptron
from sklearn.datasets import load_svmlight_file, load_breast_cancer, load_digits
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from bokeh.io import output_notebook
from bokeh.plotting import figure, show
from bokeh.models import ColumnDataSource, HoverTool
from bokeh.palettes import viridis
output_notebook()

Loading BokehJS ...

In [4]:
d = load_digits()
x = d.data
y = d.target
x_train, x_val, y_train, y_val = train_test_split(x, y, test_size=0.30, random_state=42)

In [3]:
x, y = load_svmlight_file('Datasets/mnist.scale')
x = x.toarray()
y = y.astype(int)

x_train, x_val, y_train, y_val = train_test_split(x, y, test_size=0.50, random_state=42)

In [5]:
x_test, y_test = load_svmlight_file('Datasets/mnist.scale.t')
x_test = x_test.toarray()
x_test = np.hstack((x_test, np.zeros((x_test.shape[0], 2))))
y_test = y_test.astype(int)

In [6]:
ECOC = []
c = 0
B = np.power(2, 15)
for i in range(B):
    if i%(np.ceil(B/10)) == 0:
        c += 1
        ECOC.append(np.binary_repr(i, width=15))
codes = []

for elem in ECOC:
    line = list(elem)
    codes.append(line)

new_y = []
for i in range(len(y_train)):
    new_y.append(codes[y_train[i]])
new_y = np.array(new_y, dtype=int)
new_y[new_y == 0] = -1

In [7]:

classifiers = []
for i in range(15):
    clf = kernel_perceptron(T=10)
    classifiers.append(clf)

for i in range(15):
    print(classifiers[i].train(x_train, new_y[:, i]))
    print("Classifier {} trained".format(i))


(0.9809069212410502, 0.9790322422736736, 0.9822006313559769, 0.9806088775730527)
Classifier 0 trained
(0.9689737470167065, 0.9581673115902927, 0.9639278363942317, 0.9610339419083069)
Classifier 1 trained
(0.9904534606205251, 0.9879032058890483, 0.9879032058890483, 0.9878982059143543)
Classifier 2 trained
(0.984884645982498, 0.9824902532589446, 0.9805825052314077, 0.9815304522841193)
Classifier 3 trained
(0.9824980111376292, 0.9788461350221898, 0.9788461350221898, 0.9788411350477298)
Classifier 4 trained
(0.9689737470167065, 0.9581673115902927, 0.9639278363942317, 0.9610339419083069)
Classifier 5 trained
(0.9904534606205251, 0.9879032058890483, 0.9879032058890483, 0.9878982059143543)
Classifier 6 trained
(0.984884645982498, 0.9824902532589446, 0.9805825052314077, 0.9815304522841193)
Classifier 7 trained
(0.9824980111376292, 0.9788461350221898, 0.9788461350221898, 0.9788411350477298)
Classifier 8 trained
(0.9689737470167065, 0.9581673115902927, 0.9639278363942317, 0.9610339419083069)
Cla

In [15]:
pred = np.zeros((x_val.shape[0], 15))
for i in range(15):
    pred[:, i] = classifiers[i].test(x_val)
    print("Classifier {} done".format(i))

Classifier 0 done
Classifier 1 done
Classifier 2 done
Classifier 3 done
Classifier 4 done
Classifier 5 done
Classifier 6 done
Classifier 7 done
Classifier 8 done
Classifier 9 done
Classifier 10 done
Classifier 11 done
Classifier 12 done
Classifier 13 done
Classifier 14 done


In [16]:
pred = np.array(pred)
pred[pred == -1] = 0
new_y[new_y == -1] = 0

In [18]:
code_array = np.array(codes, dtype=int)
y_pred = []
for i in range(pred.shape[0]):
    hd = []
    for j in range(len(codes)):
        hd.append(np.sum(np.abs(code_array[j] - pred[i])))
    y_pred.append(np.argmin(hd))

In [19]:
calculate_accuracy(y_pred, y_val)

0.9518518518518518

In [20]:
plot = figure(
    title='Misclassifications',
    x_axis_label='Epochs',
    y_axis_label='Misclassifications'
)

hover = HoverTool(tooltips=[('Iteration', '@itr'), ('Misclassifications', '@err')])
colors = viridis(len(classifiers))
source = []

for i in range(len(classifiers)):
    source.append(ColumnDataSource(dict(
        itr=np.linspace(0, classifiers[i].itr, classifiers[i].itr+1),
        err=classifiers[i].mis
    )))

for i in range(len(classifiers)):
    plot.line(x='itr', y='err', source=source[i], legend_label='classifier {}'.format(i+1), line_width=2, color=colors[i])
    plot.circle(x='itr', y='err', source=source[i], color=colors[i])
plot.add_tools(hover)
show(plot)